# MPV17

Pathogenic variants in ISCA2 cause [Mitochondrial DNA depletion syndrome 6 (hepatocerebral type)](https://omim.org/entry/256810). This notebook contains information from several publications.	

Note we infer that IVS3+1 G>T in PMID:22824774 is actually NM_002437.5(MPV17):c.279+1G>T (Allele ID 679728)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from IPython.display import display, HTML
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import CohortValidator
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.15


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_excel('input/MPV17_curation.xlsx')
#df.head(2)

In [4]:
df['patient_id'] = df['ID']
df.set_index('patient_id', inplace=True)

In [5]:
def extract_cdna(variant):
    """
    split strings like c.772G>T(p.Gly258*) on the open-parenthesis symbol and return the first part
    """
    v = variant.split("(")[0]
    v = v.replace(" ", "").replace("p.","")
    return v
    
def extract_variant_1(variants):
    """
    Split on the slash ("/") and return the first part (or entire string for homozygous)
    """
    v1 = variants.split("/")[0]
    return extract_cdna(v1)

def extract_variant_2(variants):
    """
    Split on the slash ("/") and return the second part (or entire string for homozygous)
    """
    fields = variants.split("/")
    if len(fields) == 2:
        return extract_cdna(fields[1])
    else:
        # there was only one variant
        return extract_cdna(variants)
        
df["var1"] = df['Variant annotation'].apply(lambda x: extract_variant_1(x))
df["var2"] = df['Variant annotation'].apply(lambda x: extract_variant_2(x))

In [6]:
from time import sleep
var1_list = df["var1"].unique()
var2_list = df["var2"].unique()
var_set = set()
var_set.update(var1_list)
var_set.update(var2_list)
variant_d = {}
hg38 = "hg38"
mpv17_transcript = "NM_002437.5"
vvalidator = VariantValidator(genome_build=hg38, transcript=mpv17_transcript)
for v in var_set:
    print(f"{v}")
    try:
        if v == "1.5-kb deletion that spanned from intron 7 well into exon 8":
            var = StructuralVariant.chromosomal_deletion(cell_contents="1.5-kb deletion that spanned from intron 7 well into exon 8",
                                                        gene_id="HGNC:26827", gene_symbol="MPV17L")
        else:
            var = vvalidator.encode_hgvs(v)
        variant_d[v] = var
    except Exception as excpt:
        print(str(excpt))
print(f"extracted {len(variant_d)} variants with VariantValidator")

c.451dupC
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.451dupC/NM_002437.5?content-type=application%2Fjson
c.279+1G>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.279+1G>T/NM_002437.5?content-type=application%2Fjson
c.191C>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.191C>G/NM_002437.5?content-type=application%2Fjson
c.186+2T>C
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.186+2T>C/NM_002437.5?content-type=application%2Fjson
c.106C>T
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.106C>T/NM_002437.5?content-type=application%2Fjson
c.408T>G
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3Ac.408T>G/NM_002437.5?content-type=application%2Fjson
c.135delA
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_002437.5%3

In [20]:
variant_d["1.5-kbdeletionthatspannedfromintron7wellintoexon8"] = StructuralVariant.chromosomal_deletion(cell_contents="1.5-kb deletion that spanned from intron 7 well into exon 8",
                                                        gene_id="HGNC:26827", gene_symbol="MPV17L")

In [7]:
import math
def decode_age_of_dx(dx_age):
    if dx_age == "newborn":
        return "P1D"
    fields = dx_age.split()
    if len(fields) != 2:
        raise ValueError(f"Bad formed{dx_age}")
    time_elem = fields[1]
    num = fields[0]
    if time_elem == "months" or time_elem == "monhts" or time_elem == "montns": 
        if "." in num:
            numbers = num.split('.')
            if numbers[1] == '5':
                return f"P{numbers[0]}M2W"
            else:
               ValueError(f"could not find parse months \"{num}\"") 
        else:
            m = int(num)
            if m > 11:
                y = math.floor(m/12)
                m = m-12*y
                return f"P{y}Y{m}M"
            else:
                return f"P{num}M"
    elif time_elem == "years":
        return f"P{num}Y"
    elif time_elem == "weeks" or time_elem == "week":
        if num == "7":
            return "P1M3W"
        else:
            return f"P{num}W"
    elif time_elem == "days" or time_elem == "day":
        return f"P{num}D"
    else:
        raise ValueError(f"could not find formed{dx_age}")

df["age"] = df['Age of diagnosis'].transform(lambda x: decode_age_of_dx(x)) 

In [8]:
ageEncoder = AgeColumnMapper.iso8601(column_name="age")
#ageEncoder.preview_column(df["age"])

In [9]:
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", unknown_symbol="nan", column_name="Gender")
#sexMapper.preview_column(df['Gender']).head()

In [10]:
mapper_d = {}
phenotypeColumnMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d={})
phenotypeColumnMapper.preview_column(df['Phenotype'])
mapper_d['Phenotype'] = phenotypeColumnMapper
#phenotypeColumnMapper.preview_column(df['Phenotype'])

In [11]:
prenatalUSmapper =  OptionColumnMapper(concept_recognizer=hpo_cr, option_d={})
prenatalUSmapper.preview_column(df['Prenatal ultrasound phenotype'])
mapper_d['Prenatal ultrasound phenotype'] = prenatalUSmapper

In [12]:
mriMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d={})
mriMapper.preview_column(df['MRI phenotype'])
mapper_d['MRI phenotype'] = mriMapper

In [13]:
# No data for cardiac
#cardiacMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d={})
#cardiacMapper.preview_column(df['Cardiac phenotype'])
#mapper_d['Cardiac phenotype'] = cardiacMapper

In [14]:
aod_d = {
    "6 years": "P6Y",
    "5 years": "P5Y",
     "4 years": "P4Y",
      "3 years": "P3Y",
    "4 months": "P4M",
    "1 month": "P1M",
    "14 months": "P1Y2M",
    "15 months": "P1Y3M",
    "6 monhts": "P6M",
    "6 months": "P6M",
    "8 months": "P8M",
    "6.5 months": "P6M2W",
     "4.5 months": "P4M2W",
       "7.5 months": "P7M2W",
    "9 months": "P9M",
    "12 months": "P1Y",
    "5 months": "P5M",
    "11 months": "P11M",
    "15 months": "P1Y3M",
     "16 months": "P1Y4M",
    "22 months": "P1Y10M",
    "30 months": "P2Y6M",
     "28 months": "P2Y4M",
}
aodMapper = AgeOfDeathColumnMapper(column_name='Age at death', string_to_iso_d=aod_d)

In [15]:
mdds6 = Disease(disease_id="OMIM:256810", disease_label="Mitochondrial DNA depletion syndrome 6 (hepatocerebral type)")
disease_d = {"256810": mdds6}
diseaseMapper = DiseaseIdColumnMapper(column_name="omim_id", disease_id_map=disease_d)

In [16]:
encoder = MixedCohortEncoder(df=df,
                            hpo_cr=hpo_cr,
                             column_mapper_d=mapper_d,
                             individual_column_name="patient_id",
                             disease_id_mapper=diseaseMapper,
                             pmid_column="PMID",
                             title_column="title",
                             sexmapper=sexMapper,
                             agemapper=ageEncoder,
                             age_of_death_mapper=aodMapper,
                             metadata=metadata
                        )

In [21]:
individuals = encoder.get_individuals()

In [22]:
# retrieve the variant strings and add Variant objects to each individual
# the individual id (i.id) is also the index of the pandas dataframe
for i in individuals:
    row = df.loc[i.id] 
    v1 = row['var1']
    v2 = row['var2']
    
    #print(f"{i.id}: v1={v1} and v2={v2}")
    if v1 == v2:
        var1 = variant_d.get(v1)
        var1.set_homozygous()
        i.add_variant(var1)
    else:
        var1 = variant_d.get(v1)
        var2 = variant_d.get(v2)
        var1.set_heterozygous()
        var2.set_heterozygous()
        i.add_variant(var1)
        i.add_variant(var2)


In [23]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,6


In [24]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
34979697_P1 (FEMALE; P1Y6M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.293C>T (homozygous),Distal lower limb muscle weakness (HP:0009053); Motor regression (HP:0033044); Mental deterioration (HP:0001268); Positive Romberg sign (HP:0002403); Impaired distal tactile sensation (HP:0006937); Foot dorsiflexor weakness (HP:0009027); Lower limb hyperreflexia (HP:0002395); Increased serum lactate (HP:0002151); Hyperammonemia (HP:0001987); Increased CSF protein concentration (HP:0002922); Abnormal cerebral white matter morphology (HP:0002500)
34035203_P1 (FEMALE; P1Y5M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.451dup (heterozygous) NM_002437.5:c.293C>T (heterozygous),Small for gestational age (HP:0001518); Microcephaly (HP:0000252); Delayed gross motor development (HP:0002194); Increased serum lactate (HP:0002151); Elevated hepatic transaminase (HP:0002910); Increased total bilirubin (HP:0003573); Abnormality of the coagulation cascade (HP:0003256); Increased hepatic echogenicity (HP:0031141); Hyperintensity of cerebral white matter on MRI (HP:0030890)
34023347_P1 (MALE; P1Y6M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.293C>T (homozygous),Acute hepatic failure (HP:0006554); Intellectual disability (HP:0001249)
33486010_P1 (MALE; P6M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.121C>T (homozygous),Global developmental delay (HP:0001263); Peripheral neuropathy (HP:0009830); Scoliosis (HP:0002650); Cholestasis (HP:0001396); Hepatomegaly (HP:0002240); Hypoglycemia (HP:0001943); Cirrhosis (HP:0001394)
31664948_P1 (FEMALE; P4M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.461+1G>C (homozygous),Poor suck (HP:0002033); Seizure (HP:0001250); Weak cry (HP:0001612); Failure to thrive (HP:0001508); Muscle weakness (HP:0001324); Jaundice (HP:0000952); Hypertyrosinemia (HP:0003231); Cirrhosis (HP:0001394); Elevated hepatic transaminase (HP:0002910); Neurodevelopmental delay (HP:0012758)
31664948_P2 (MALE; P5M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.277C>T (homozygous),Malaise (HP:0033834); Prolonged neonatal jaundice (HP:0006579); Dark urine (HP:0040319); Poor suck (HP:0002033); Neurodevelopmental delay (HP:0012758); Elevated hepatic transaminase (HP:0002910); Hyperbilirubinemia (HP:0002904); Hypertyrosinemia (HP:0003231); Hyperphenylalaninemia (HP:0004923)
29318572_P1 (FEMALE; P4M),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.106C>T (homozygous),Failure to thrive (HP:0001508); Jaundice (HP:0000952); Hypoglycemia (HP:0001943); Infantile axial hypotonia (HP:0009062); Muscle weakness (HP:0001324); Hepatomegaly (HP:0002240); Elevated hepatic transaminase (HP:0002910); Hepatic steatosis (HP:0001397); Conjugated hyperbilirubinemia (HP:0002908); Increased serum lactate (HP:0002151); Hypoalbuminemia (HP:0003073); Abnormality of the coagulation cascade (HP:0003256); Intrahepatic cholestasis (HP:0001406); Periportal fibrosis (HP:0001405)
29318572_P2 (FEMALE; P6M2W),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.106C>T (homozygous),Severe failure to thrive (HP:0001525); Jaundice (HP:0000952); Hypoglycemia (HP:0001943); Hepatosplenomegaly (HP:0001433); Hypotonia (HP:0001252); Hyporeflexia (HP:0001265); Elevated hepatic transaminase (HP:0002910); Conjugated hyperbilirubinemia (HP:0002908); Increased serum lactate (HP:0002151); Abnormality of the coagulation cascade (HP:0003256)
28673863_P1 (FEMALE; P11Y),Mitochondrial DNA depletion syndrome 6 (hepatocerebral type) (OMIM:256810),NM_002437.5:c.121C>T (homozygous),Myalgia (HP:0003326); Lower limb muscle weakness (HP:0007340); Postural instability (HP:0002172); Vomiting (HP:0002013); Abdominal distention (HP:

In [25]:
MixedCohortEncoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 50 GA4GH phenopackets to the directory phenopackets
